In [37]:
# !pip install ipywidgets==7.6.0

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# !pip install smdebug

In [3]:
from datetime import datetime


In [4]:
import sagemaker
import boto3

from sagemaker.pytorch import PyTorch
from sagemaker import get_execution_role

from sagemaker.tuner import (
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)

In [5]:
sess = sagemaker.Session()
bucket = sess.default_bucket()
prefix = "project-3-cola"
role = get_execution_role()

In [6]:
now = str(datetime.now())

estimator = PyTorch(
    entry_point="model_hpo.py",
    source_dir="../src",
    output_path=f"s3://{bucket}/{prefix}",
    checkpoint_s3_uri=f"s3://{bucket}/{prefix}/checkpoints/{now}",
    base_job_name="hyperparametere-tuning-cola",
    role=role,
    instance_count=1,
    instance_type="ml.g4dn.xlarge",
    framework_version="1.10",
    py_version="py38",

)

In [7]:
hyperparameter_ranges = {
    "lr": ContinuousParameter(2e-5, 2e-4),
    "batch-size": CategoricalParameter([16, 32, 64]),
    "max-length": CategoricalParameter([128, 256])
}

In [8]:
objective_metric_name = "Average Validation Loss"
objective_type = "Minimize"
metric_definitions = [{"Name": "Average Validation Loss", "Regex": "Validation set: Average loss: ([0-9\\.]+)"}]

In [9]:
tuner = HyperparameterTuner(
    estimator,
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitions,
    max_jobs=2,
    max_parallel_jobs=2,
    objective_type=objective_type,
)

In [10]:
tuner.fit(wait=True)

................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................!


In [11]:
tuner.best_training_job()

'pytorch-training-230930-1009-001-673f1072'

In [14]:
tuner.best_estimator().hyperparameters()


2023-09-30 10:54:49 Starting - Preparing the instances for training
2023-09-30 10:54:49 Downloading - Downloading input data
2023-09-30 10:54:49 Training - Training image download completed. Training in progress.
2023-09-30 10:54:49 Uploading - Uploading generated training model
2023-09-30 10:54:49 Completed - Resource released due to keep alive period expiry


{'_tuning_objective_metric': '"Average Validation Loss"',
 'batch-size': '"64"',
 'lr': '0.00015571726483079338',
 'max-length': '"256"',
 'sagemaker_container_log_level': '20',
 'sagemaker_estimator_class_name': '"PyTorch"',
 'sagemaker_estimator_module': '"sagemaker.pytorch.estimator"',
 'sagemaker_job_name': '"hyperparametere-tuning-cola-2023-09-30-10-09-01-165"',
 'sagemaker_program': '"model_hpo.py"',
 'sagemaker_region': '"us-east-1"',
 'sagemaker_submit_directory': '"s3://sagemaker-us-east-1-046017406246/hyperparametere-tuning-cola-2023-09-30-10-09-01-165/source/sourcedir.tar.gz"'}